In [28]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime
from time import sleep
from random import randint

# API Camp to Camp - 

In [8]:
#Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        print(api_path)
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))

https://api.camptocamp.org/outings?act=skitouring&a=14274
https://api.camptocamp.org/outings?act=rock_climbing&a=14274
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=30
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274
Nombre total de reqûetes :4
4
[{'total': 52762, 'documents': [{'activities': ['skitouring'], 'img_count': 5, 'date_start': '2020-03-23', 'locales': [{'lang': 'fr', 'version': 2, 'summary': None, 'title': 'Mont Mirantin : Face NW - depuis Molliessoulaz de loin '}], 'available_langs': ['fr'], 'labande_global_rating': 'TD-', 'author': {'user_id': 1071965, 'name': 'christian73'}, 'document_id': 1196222, 'date_end': '2020-03-23', 'condition_rating': 'excellent', 'protected': False, 'quality': 'fine', 'areas': [{'version': 11, 'locales': [{'lang': 'ca', 'version': 3, 'title': 'França'}, {'lang': 'de', 'version': 1, 'title': 'Frankreich'}, {'lang': 'en', 'version': 1, 'title': 'France'}, {'lang': 'es', 'version': 1, 'title': 'Francia'}, {'lan

# # Build dataframe & export to csv - 

In [53]:
# Flatten json file - 
d = pd.DataFrame()

for k in range(len(json_data)):    
    d=d.append(json_normalize(json_data[k]['documents'],max_level=1))

d.reset_index(inplace=True, drop=True)

# Select relevant columns -  
cols = ['document_id','activities','date_start','date_end','author.name','author.user_id']
d=d[cols]

# Print first rows
d.head()

,document_id,activities,date_start,date_end,author.name,author.user_id
0,1196222,[skitouring],2020-03-23,2020-03-23,christian73,1071965
1,1195244,"[skitouring, snow_ice_mixed, mountain_climbing...",2020-03-17,2020-03-17,dormeur10,800213
2,1195133,[skitouring],2020-03-17,2020-03-17,christian73,1071965
3,1195132,[skitouring],2020-03-17,2020-03-17,janic,578085
4,1195116,"[skitouring, snow_ice_mixed, paragliding]",2020-03-17,2020-03-17,Paulo,991164


In [54]:
# Clean columns -

# Clean activity - 
def dummies_activity(x,a):
    if a in x:
        return True
    else:
        return False

activities=['skitouring', 'rock_climbing','mountain_climbing','ice_climbing','hiking','snow_ice_mixed']

for activity in activities:
    d[activity]=d.activities.apply(lambda x: dummies_activity(x,activity))

d.drop('activities', inplace=True, axis=1)

# Drop_duplicates (caused by multi-activities outings exported several times during requests)
d.drop_duplicates(inplace=True)

# Convert to datetime
col_to_date=['date_end','date_start']
d[col_to_date]=d[col_to_date].apply(lambda x: pd.to_datetime(x))

# Convert to integer
col_to_int=['document_id','author.user_id']
d[col_to_int]=d[col_to_int].astype(int)

# Export to csv -
d.to_csv('outings.csv',index=False)

# CHECK - 
d.head()

,document_id,date_start,date_end,author.name,author.user_id,skitouring,rock_climbing,mountain_climbing,ice_climbing,hiking,snow_ice_mixed
0,1196222,2020-03-23,2020-03-23,christian73,1071965,True,False,False,False,False,False
1,1195244,2020-03-17,2020-03-17,dormeur10,800213,True,False,True,True,False,True
2,1195133,2020-03-17,2020-03-17,christian73,1071965,True,False,False,False,False,False
3,1195132,2020-03-17,2020-03-17,janic,578085,True,False,False,False,False,False
4,1195116,2020-03-17,2020-03-17,Paulo,991164,True,False,False,False,False,True
